In [100]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1) Load the training data into feature matrix, class labels, and event ids:

In [103]:
from proj1_helpers import *
from helpers import *
from data_modification import replace_by_mean

""" Load TRAINING data """
DATA_TRAIN_PATH = '../data/train.csv'
y, raw_tx, ids = load_csv_data(DATA_TRAIN_PATH)

# Replace -999 by the mean of its respective column
processed_tx = replace_by_mean(raw_tx)

# Standardize (subtract mean and divive by standard deviation)
processed_tx,_,_ = standardize(processed_tx)

""" Load TEST data """
DATA_TEST_PATH = '../data/test.csv' # download test data and supply path here 
_, raw_tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

processed_tx_test = replace_by_mean(raw_tX_test)
processed_tx_test,_,_ = standardize(processed_tx_test)


## 2) Regular Ridge regression :

In [102]:
from run_ridge_regression import *

tx = np.copy(processed_tx)

lambdas=np.logspace(-10, 10, 30)
k_fold=10
plot_filename = "regular_ridge_regression"

run_ridge_regression(y, tx, lambdas, k_fold, filename=plot_filename)

### 3) Compute weights and adapt test data for submission

In [104]:
from ridge_regression import ridge_regression
lambda_ = 1e-3
loss, weights = ridge_regression(y, tx, lambda_)

tx_test = np.copy(processed_tx_test)

## 2) Ridge regression with polynomial basis functions:

In [105]:
from run_ridge_regression import *
from build_polynomial import *

degrees = [2, 3, 5]
k_fold=10
lambdas=np.logspace(-10, 10, 30)

tx = np.copy(processed_tx)

for degree in degrees:
    print("Iteration for degree",degree, "started.")
    plot_filename = "ridge_regression_poly_deg_%d" % degree
    tx_poly = build_poly_matrix(np.copy(tx), degree)
    run_ridge_regression(y, tx_poly, lambdas, k_fold, filename=plot_filename)
    print("Iteration for degree",degree, "over.")

Iteration for degree 2 started.
Iteration for degree 2 over.
Iteration for degree 3 started.
Iteration for degree 3 over.
Iteration for degree 5 started.
Iteration for degree 5 over.


### 3) Compute weights and adapt test data for submission

In [106]:
from ridge_regression import ridge_regression
from build_polynomial import *

degree = 2
lambda_ = 1e-3
tx_poly = build_poly_matrix(tx, degree)
loss, weights = ridge_regression(y, tx_poly, lambda_)

tx_test = np.copy(processed_tx_test)
tx_test = build_poly_matrix(tx_test, degree)

## 2) Ridge regression, tx augmented with cos:

In [108]:
from run_ridge_regression import *
from add_functions_of_features import *

tx = np.copy(processed_tx)

k_fold=10
lambdas=np.logspace(-10, 20, 30)
plot_filename = "ridge_regression_cos"

tx = add_cos_function(tx)
run_ridge_regression(y, tx, lambdas, k_fold, filename=plot_filename)

### 3) Compute weights and adapt test data for submission

In [ ]:
from ridge_regression import ridge_regression
from add_functions_of_features import *

lambda_ = 1e-3
loss, weights = ridge_regression(y, tx, lambda_)

tx_test = np.copy(processed_tx_test)
tx_test = add_cos_function(tx_test)

## 2) Ridge regression, tx augmented with exp:

In [110]:
from run_ridge_regression import *
from add_functions_of_features import *

tx = np.copy(processed_tx)

k_fold=10
lambdas=np.logspace(-10, 20, 30)
plot_filename = "ridge_regression_exp"

tx = add_exp_function(tx)
run_ridge_regression(y, tx, lambdas, k_fold, filename=plot_filename)

### 3) Compute weights and adapt test data for submission

In [111]:
from ridge_regression import ridge_regression
from add_functions_of_features import *

lambda_ = 1e-3
loss, weights = ridge_regression(y, tx, lambda_)

tx_test = np.copy(processed_tx_test)
tx_test = add_exp_function(tx_test)

## 4) Generate predictions and save ouput in csv format for submission:

In [112]:
import datetime
now = datetime.datetime.now()
now_str = now.strftime("%d-%m-%Y_%Hh%M_%S")
OUTPUT_PATH = '../data/predictions-%s-group-36.csv' % now_str
y_pred = predict_labels(weights, tx_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)